In [ ]:
from IPython.core.display import HTML

HTML(
    """
    <style>

    .usecase-title, .usecase-duration, .usecase-section-header {
      padding-left: 15px;
      padding-bottom: 10px;
      padding-top: 10px;
      padding-right: 15px;
      background-color: #0f9295;
      color: #fff;
      }

    .usecase-title {
      font-size: 1.7em;
      font-weight: bold;
      }

    .usecase-authors, .usecase-level, .usecase-skill {
      padding-left: 15px;
      padding-bottom: 7px;
      padding-top: 7px;
      background-color: #baeaeb;
      font-size: 1.4em;
      color: #121212;
      }

    .usecase-level-skill {
      display: flex;
      }

    .usecase-level, .usecase-skill {
      width: 50%;
      }

    .usecase-duration, .usecase-skill {
      text-align: right;
      padding-right: 15px;
      padding-bottom: 8px;
      font-size: 1.4em;
      }

    .usecase-section-header {
      font-weight: bold;
      font-size: 1.5em;
      }

    .usecase-subsection-header, .usecase-subsection-blurb {
      font-weight: bold;
      font-size: 1.2em;
      color: #121212;
      }

    .usecase-subsection-blurb {
      font-size: 1em;
      font-style: italic;
      }

    </style>
    """)

### **Support Services**

**Authored by**: Thakoor Madhuvaishali

**Duration:**

**Level:** Intermediate

**Pre-requisite Skills:** Python, Data analysis, data cleaning, data visualisation,

Scenario:

As a user in need of support services, I want to be able to easily find and locate nearby support services, so that I can access the help I need in a timely manner. I may be facing various challenges such as financial difficulties, mental health issues, or lack of access to transportation, and having easy access to support services can significantly improve my well-being and quality of life.

What this use case will teach you

At the end of this use case, you will:

*   Learn how to use Python to analyze and visualize data, enabling you to extract meaningful insights from datasets related to support services and public transport.
*   Gain experience in mapping and geocoding, allowing you to plot support service locations on a map and determine the nearest public transport options.
*   Understand how to integrate multiple datasets for analysis, helping you combine information about support services, bus stops, and tram tracks to create a comprehensive and user-friendly tool for accessing support services.

Introduction:

Accessing support services can be challenging for many individuals, especially those facing financial difficulties or transportation barriers. In such situations, having a centralized and easily accessible platform to locate support services can make a significant difference. The Support Services Locator website aims to bridge this gap by providing a user-friendly interface that allows users to search for support services based on their specific needs and location.

The Support Services use case aims to address the challenge of accessing support services for individuals in need, particularly those facing financial constraints or transportation difficulties. By creating a website with a map filter function, users can easily find nearby support services such as counseling, food assistance, and healthcare. The website will also display the nearest bus stop or tram station to help users reach their destination efficiently.

**DATASETS LIST:**
- Free and cheap support services, with opening hours, public transport, and parking options

This dataset contains information about various support services available to the community and includes details such as the name and address of each service, their opening hours, the availability of public transport options (e.g., bus stops, tram stations) near the location, and information about parking availability.

- Bus stops

This dataset contains information about the locations of bus stops in the area, including their geographic coordinates (latitude and longitude). This information is crucial for helping users navigate to their desired support service locations using public transport.

- Tram tracks

This dataset contains information about the tram tracks in the area, including their routes and stops. Tram tracks are another important aspect of public transport infrastructure that can help users plan their journeys to support service locations.








# **Part 1 - Import modules and fetch data from Open Data Melbourne**

---



In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap

# Function that extracts the data from the JSON
def get_data(base, data_url, offset = 0):

    filters = f'records?limit={100}&offset={offset}&timezone=UTC'
    url = f'{base}{data_url}/{filters}'
    result = requests.get(url)
    # if you need to extract more than 9900 records, enter the api key like below
    #result = requests.get(url + f'&apikey={API_KEY}')
    result_json = result.json()
    max_results = result_json['total_count']
    links = result_json['links']
    records = result_json['records']
    records_df = pd.json_normalize(records)

    # fix columns names
    records_df.drop(columns=['links'],inplace=True)
    column_names = records_df.columns.values.tolist()

    column_names = [i.split('.')[-1] for i in column_names]
    records_df.columns = column_names

    next_url = None

    # get next url
    if records_df.shape[0] != max_results:
        for l in links:
            if l['rel']=='next':
                next_url = l['href']

    return [records_df, next_url, column_names]

In [ ]:
# Appends each set of data
def add_to_dataframe(dataframe, new_data):
    df = pd.concat([dataframe, new_data])
    return df

In [ ]:
# Extracts the data from the url to produce a pandas dataset
def extract_dataframe(dataset):
    df_new, url_next, column_names = get_data(base_url, dataset)
    df = pd.DataFrame(columns=column_names)
    while url_next != None:
        df_new, url_next, column_names = get_data(base_url, dataset, df.shape[0])
        df = add_to_dataframe(df, df_new)
        # This limits the dataset to only 9900 entries, should be removed if you want to import more data
        # To get more than 9900, enter the API key in the get data function
        if df.shape[0] == 9900:
            break
    return df

In [ ]:
# Base url for v2 api
base_url = 'https://data.melbourne.vic.gov.au/api/explore/v2.0/catalog/datasets/'

# A filter at the end of the url
number_of_records = 100
filters = f'records?limit={number_of_records}&offset=0&timezone=UTC'

# The generated url
# dataset_url = f'{base_url}{dataset}/{filters}'

In [ ]:
# Call the function and produce a dataframe
df = extract_dataframe("free-and-cheap-support-services-with-opening-hours-public-transport-and-parking-")


In [ ]:
df_bus_stop = extract_dataframe("bus-stops")
df_tram_tracks = extract_dataframe("tram-tracks")

In [ ]:
df.head(20) # head of rows in the dataset

,id,timestamp,size,name,what,who,address_1,address_2,suburb,phone,...,category_2,category_3,category_4,category_5,category_6,longitude,latitude,lon,lat,geocoded_location
0,ffb55e9ec8257828aa5057ebfc22f5eb9f5d18d2,2022-11-13T10:02:12.681Z,764,Action Centre,"contraception, sexuality and gender issues, pr...",Services are for under 25 year olds,Level 1,92-94 Elizabeth Street,Melbourne,9660 4700,...,Counselling and Psychiatric Services,N/A,N/A,N/A,N/A,144.964170,-37.815731,144.964170,-37.815731,NaN
1,c6796c9abf80e8dfd45eec88b9b0a169c03f0b48,2022-11-13T10:02:12.681Z,839,Anglicare Victoria – Mission House,Provides Emergency Relief to people living in ...,Assistance may be given to those living in the...,None,122 Napier Street,Fitzroy,9486 0445,...,Clothes and Blankets,Showers / Laundry,N/A,N/A,N/A,144.979960,-37.804544,144.979960,-37.804544,NaN
2,184866856a1e5cfb558bd0f314e79ba809c4c3cb,2022-11-13T10:02:12.681Z,790,Australian College of Optometry Outreach Services,"Homeless Persons Eyecare Program, Supported Re...",Low cost eye care for those of limited means. ...,None,Corner Keppel and Cardigan Streets,Carlton,9349 7400,...,N/A,N/A,N/A,N/A,N/A,NaN,NaN,NaN,NaN,NaN
3,bd769f1c88b728a2987ec860f4ca0381ce7d8ed3,2022-11-13T10:02:12.681Z,586,CatholicCare/St Mary of the Cross Centre,"referral, drug education, counselling, group s...",None,None,23 Brunswick Street,Fitzroy,8417 1200,...,Counselling and Psychiatric Services,N/A,N/A,N/A,N/A,144.976675,-37.806595,144.976675,-37.806595,NaN
4,9da019f0c81bd00463c356930940e258cc16bf49,2022-11-13T10:02:12.681Z,960,Central City Community Health Service,Outreach support services for people experienc...,None,None,53 Victoria Street,Melbourne,1300 022 247,...,Counselling and Psychiatric Services,Legal / Financial Advice,N/A,N/A,N/A,144.961214,-37.806518,144.961214,-37.806518,NaN
5,ba6a12d05c816ef7ce98d5de9dac7123927759d0,2022-11-13T10:02:12.681Z,567,Centrelink,Centrelink is an Australian Government agency ...,None,None,94 York Street,South Melbourne,132850,...,Employment Assistance,Tenancy Assistance,N/A,N/A,N/A,144.958914,-37.830941,144.958914,-37.830941,NaN
6,675fef59f085cc5ed54b5d73af21409277fa9dea,2022-11-13T10:02:12.681Z,446,Clarendon Clinic,psychiatric services for people with serious m...,None,None,52 Albert Street,East Melbourne,9417 5696,...,N/A,N/A,N/A,N/A,N/A,144.988927,-37.810921,144.988927,-37.810921,NaN
7,e5bfec0e7180e746ecf5c9fdcd16e4a124927a2e,2022-11-13T10:02:12.681Z,577,COHEALTH,broad range of allied health and counselling s...,None,None,622 Lygon Street,Carlton North,9349 7333,...,Needle Exchange,N/A,N/A,N/A,N/A,144.968594,-37.791715,144.968594,-37.791715,NaN
8,af28916f2ea4de8ec829fbcfe7413039649a8997,2022-11-13T10:02:12.681Z,836,Frontyard Youth Services,"accommodation referral, centrelink, youth conn...","aged between 12 to 25\nhomeless, at risk of ho...",None,19 King Street,Melbourne,9611 2411,...,Health Services / Pharmacy,Legal / Financial Advice,Employment Assistance,N/A,N/A,144.957305,-37.819928,144.957305,-37.819928,NaN
9,645267fa734a80f4ce0699f011002401eb193b17,2022-11-13T10:02:12.681Z,726,Hanover Fitzroy,The Assertive Outreach Team offers support to ...,None,None,145 Smith Street,Fitzroy,9288 9800,...,Health Services / Pharmacy,Clothes and Blankets,N/A,N/A,N/A,144.982736,-37.804578,144.982736,-37.804578,NaN


# **Exploring and Cleaning of the dataset**

In [ ]:
df.info() # number of rows and columns in the dataset

<class 'pandas.core.frame.DataFrame'>
Index: 119 entries, 0 to 18
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     119 non-null    object 
 1   timestamp              119 non-null    object 
 2   size                   119 non-null    object 
 3   name                   118 non-null    object 
 4   what                   97 non-null     object 
 5   who                    27 non-null     object 
 6   address_1              17 non-null     object 
 7   address_2              90 non-null     object 
 8   suburb                 90 non-null     object 
 9   phone                  100 non-null    object 
 10  phone_2                4 non-null      object 
 11  free_call              27 non-null     object 
 12  email                  50 non-null     object 
 13  website                114 non-null    object 
 14  twitter                62 non-null     object 
 15  social_media

In [ ]:
#Find the missing values
df.isnull().sum()


id                         0
timestamp                  0
size                       0
name                       1
what                      22
who                       92
address_1                102
address_2                 29
suburb                    29
phone                     19
phone_2                  115
free_call                 92
email                     69
website                    5
twitter                   57
social_media              58
monday                    31
tuesday                   31
wednesday                 31
thursday                  31
friday                    31
saturday                  32
sunday                    32
public_holidays           32
cost                     103
tram_routes               44
bus_routes               112
nearest_train_station     87
category_1                 1
category_2                 1
category_3                 1
category_4                 1
category_5                 1
category_6                 1
longitude     

In [ ]:
df = df.dropna(subset=['latitude', 'longitude', 'name'])

In [ ]:
# Check for missing values in latitude and longitude columns
missing_lat = df['latitude'].isnull().sum()
missing_lon = df['longitude'].isnull().sum()

# Print the number of missing values
print(f"Missing latitude values: {missing_lat}")
print(f"Missing longitude values: {missing_lon}")

Missing latitude values: 0
Missing longitude values: 0


In [ ]:
df.head(5)

,id,timestamp,size,name,what,who,address_1,address_2,suburb,phone,...,category_2,category_3,category_4,category_5,category_6,longitude,latitude,lon,lat,geocoded_location
0,ffb55e9ec8257828aa5057ebfc22f5eb9f5d18d2,2022-11-13T10:02:12.681Z,764,Action Centre,"contraception, sexuality and gender issues, pr...",Services are for under 25 year olds,Level 1,92-94 Elizabeth Street,Melbourne,9660 4700,...,Counselling and Psychiatric Services,N/A,N/A,N/A,N/A,144.964170,-37.815731,144.964170,-37.815731,NaN
1,c6796c9abf80e8dfd45eec88b9b0a169c03f0b48,2022-11-13T10:02:12.681Z,839,Anglicare Victoria – Mission House,Provides Emergency Relief to people living in ...,Assistance may be given to those living in the...,None,122 Napier Street,Fitzroy,9486 0445,...,Clothes and Blankets,Showers / Laundry,N/A,N/A,N/A,144.979960,-37.804544,144.979960,-37.804544,NaN
3,bd769f1c88b728a2987ec860f4ca0381ce7d8ed3,2022-11-13T10:02:12.681Z,586,CatholicCare/St Mary of the Cross Centre,"referral, drug education, counselling, group s...",None,None,23 Brunswick Street,Fitzroy,8417 1200,...,Counselling and Psychiatric Services,N/A,N/A,N/A,N/A,144.976675,-37.806595,144.976675,-37.806595,NaN
4,9da019f0c81bd00463c356930940e258cc16bf49,2022-11-13T10:02:12.681Z,960,Central City Community Health Service,Outreach support services for people experienc...,None,None,53 Victoria Street,Melbourne,1300 022 247,...,Counselling and Psychiatric Services,Legal / Financial Advice,N/A,N/A,N/A,144.961214,-37.806518,144.961214,-37.806518,NaN
5,ba6a12d05c816ef7ce98d5de9dac7123927759d0,2022-11-13T10:02:12.681Z,567,Centrelink,Centrelink is an Australian Government agency ...,None,None,94 York Street,South Melbourne,132850,...,Employment Assistance,Tenancy Assistance,N/A,N/A,N/A,144.958914,-37.830941,144.958914,-37.830941,NaN


In [ ]:
map = folium.Map(location=[-37.8136, 144.9631], zoom_start=12)  # Melbourne coordinates
for index, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(map)


In [ ]:
map

In [ ]:
df_bus_stop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 309 entries, 0 to 8
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           309 non-null    object 
 1   timestamp    309 non-null    object 
 2   size         309 non-null    object 
 3   lon          309 non-null    float64
 4   lat          309 non-null    float64
 5   type         309 non-null    object 
 6   coordinates  309 non-null    object 
 7   type         309 non-null    object 
 8   prop_id      309 non-null    object 
 9   addresspt1   309 non-null    float64
 10  addressp_1   309 non-null    object 
 11  asset_clas   309 non-null    object 
 12  asset_type   309 non-null    object 
 13  objectid     309 non-null    object 
 14  str_id       309 non-null    object 
 15  addresspt    309 non-null    object 
 16  asset_subt   0 non-null      object 
 17  model_desc   309 non-null    object 
 18  mcc_id       309 non-null    object 
 19  roadseg_id   30

In [ ]:
df_tram_tracks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 645 entries, 0 to 44
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           645 non-null    object 
 1   timestamp    645 non-null    object 
 2   size         645 non-null    object 
 3   lon          645 non-null    float64
 4   lat          645 non-null    float64
 5   type         645 non-null    object 
 6   coordinates  645 non-null    object 
 7   type         645 non-null    object 
 8   descriptio   645 non-null    object 
 9   name         645 non-null    object 
dtypes: float64(2), object(8)
memory usage: 55.4+ KB


In [ ]:
#Find the missing values
df_bus_stop.isnull().sum()

id               0
timestamp        0
size             0
lon              0
lat              0
type             0
coordinates      0
type             0
prop_id          0
addresspt1       0
addressp_1       0
asset_clas       0
asset_type       0
objectid         0
str_id           0
addresspt        0
asset_subt     309
model_desc       0
mcc_id           0
roadseg_id       0
descriptio       0
model_no         0
dtype: int64

In [ ]:
#Find the missing values
df_tram_tracks.isnull().sum()

id             0
timestamp      0
size           0
lon            0
lat            0
type           0
coordinates    0
type           0
descriptio     0
name           0
dtype: int64

In [ ]:
# Assume df_tram and df_bus are DataFrames containing tram and bus stop data respectively

# Add tram stop markers to the map
for index, row in df_tram_tracks.iterrows():
    folium.Marker([row['lat'], row['lon']], icon=folium.Icon(color='green'), popup=row['name']).add_to(map)

# Add bus stop markers to the map
for index, row in df_bus_stop.iterrows():
    folium.Marker([row['lat'], row['lon']], icon=folium.Icon(color='blue'), popup=row['descriptio']).add_to(map)

In [ ]:
map


In [ ]:
pip install folium --upgrade


In [ ]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Create a map object
map = folium.Map(location=[-37.8136, 144.9631], zoom_start=12)

# Add support service markers to the map
marker_cluster = MarkerCluster().add_to(map)
for index, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(marker_cluster)

# Create a filter for categories
categories = df['category_1'].unique().tolist()
category_filter = widgets.Dropdown(options=['All'] + categories, value='All', description='Category:')

def update_map(category):
    # Remove all markers from the map
    for marker in marker_cluster.get_markers():
        map.remove_layer(marker)

    # Add markers based on the selected category
    if category == 'All':
        for index, row in df.iterrows():
            folium.Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(marker_cluster)
    else:
        filtered_df = df[df['category_1'] == category]
        for index, row in filtered_df.iterrows():
            folium.Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(marker_cluster)

category_filter.observe(lambda change: update_map(change.new), names='value')

# Display the map and filter
display(category_filter)
display(map)


Dropdown(description='Category:', options=('All', 'Health Services / Pharmacy', 'Food', 'Needle Exchange', 'Le…

In [ ]:
from math import radians, sin, cos, sqrt, atan2

# Assuming df contains your support service locations DataFrame
# Assuming bus_stops_df contains your bus stop locations DataFrame

# Function to calculate distance between two points
def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = radians(lat1), radians(lon1), radians(lat2), radians(lon2)

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    r = 6371  # Radius of Earth in kilometers
    distance = r * c

    return distance

# Iterate over each support service and calculate the distance to the nearest bus stop
for index, service_row in df.iterrows():
    nearest_distance = float('inf')  # Initialize with a large value
    for stop_index, stop_row in df_bus_stop.iterrows():
        distance = haversine_distance(service_row['lat'], service_row['lon'],
                                       stop_row['lat'], stop_row['lon'])
        if distance < nearest_distance:
            nearest_distance = distance
    # Update the support service DataFrame with the nearest distance to a bus stop
    df.at[index, 'nearest_bus_stop_distance'] = nearest_distance


In [ ]:
import folium
from folium.plugins import MarkerCluster

# Create a map object
map = folium.Map(location=[-37.8136, 144.9631], zoom_start=12)

# Add support service markers to the map
service_marker_cluster = MarkerCluster().add_to(map)
for index, service_row in df.iterrows():
    folium.Marker([service_row['latitude'], service_row['longitude']], popup=service_row['name']).add_to(service_marker_cluster)

# Add nearest bus stop markers to the map
bus_stop_marker_cluster = MarkerCluster().add_to(map)
for index, service_row in df.iterrows():
    nearest_distance = float('inf')
    nearest_bus_stop = None
    for stop_index, stop_row in df_bus_stop.iterrows():
        distance = haversine_distance(service_row['lat'], service_row['lon'],
                                       stop_row['lat'], stop_row['lon'])
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_bus_stop = stop_row
    if nearest_bus_stop is not None:
        folium.Marker([nearest_bus_stop['lat'], nearest_bus_stop['lon']],
                      popup=f"Nearest Bus Stop to {service_row['name']}").add_to(bus_stop_marker_cluster)

# Display the map
map


In [ ]:
import folium
from folium.plugins import MarkerCluster

# Create a map object
map = folium.Map(location=[-37.8136, 144.9631], zoom_start=12)

# Add support service markers to the map
service_marker_cluster = MarkerCluster(name='Support Services').add_to(map)
for index, service_row in df.iterrows():
    folium.Marker([service_row['lat'], service_row['lon']],
                  popup=service_row['name'],
                  icon=folium.Icon(color='blue', icon='info-sign')).add_to(service_marker_cluster)

# Add nearest bus stop markers to the map
bus_stop_marker_cluster = MarkerCluster(name='Bus Stops').add_to(map)
for index, service_row in df.iterrows():
    nearest_distance = float('inf')
    nearest_bus_stop = None
    for stop_index, stop_row in df_bus_stop.iterrows():
        distance = haversine_distance(service_row['lat'], service_row['lon'],
                                       stop_row['lat'], stop_row['lon'])
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_bus_stop = stop_row
    if nearest_bus_stop is not None:
        folium.Marker([nearest_bus_stop['lat'], nearest_bus_stop['lon']],
                      popup=f"Nearest Bus Stop to {service_row['name']}",
                      icon=folium.Icon(color='green', icon='bus')).add_to(bus_stop_marker_cluster)

        # Draw a path from support service to bus stop
        points = [[service_row['lat'], service_row['lon']],
                  [nearest_bus_stop['lat'], nearest_bus_stop['lon']]]
        folium.PolyLine(locations=points, color='red', weight=2.5, opacity=1).add_to(map)

# Add layer control to toggle markers and paths
folium.LayerControl().add_to(map)

# Display the map
map
